In [1]:
# Cell 1: Imports and Setup (SIFT Version - Corrected Weights Path)

import os
import re
import cv2 # Use cv2 directly
import numpy as np
import torch
import matplotlib.pyplot as plt # Usually needed later for plots if enabled
from pathlib import Path
import sys
import math
import time # For timing cell later
import random # For timing cell later
import traceback # For detailed errors later

print(f"Script started/restarted: {time.strftime('%Y-%m-%d %H:%M:%S')}")

# Add the cloned repository directory to Python path
# Assumes the repo is cloned in the same directory as the notebook
superglue_sift_repo_path = Path('./SuperGlue-pytorch')
if not superglue_sift_repo_path.is_dir():
    # Try to give a more helpful error message
    print(f"ERROR: Repository directory not found at '{superglue_sift_repo_path.resolve()}'")
    print("Please ensure you have cloned 'https://github.com/yingxin-jia/SuperGlue-pytorch'")
    print("into the same directory where this notebook is located,")
    print("or update the 'superglue_sift_repo_path' variable.")
    # Raise error to stop execution if repo not found
    raise FileNotFoundError(f"Repository directory not found at {superglue_sift_repo_path}")
else:
    print(f"Found SuperGlue-pytorch repository at: {superglue_sift_repo_path.resolve()}")
    sys.path.append(str(superglue_sift_repo_path))

# Import SuperGlue model components from THIS repository
# *** NOTE: You MAY need to adjust the exact import path/class name based on the repo ***
try:
    from models.superglue import SuperGlue # Common structure, verify if needed
    print("Successfully imported SuperGlue class from repository.")
except ImportError as e:
    print(f"ERROR: Failed to import SuperGlue class from '{superglue_sift_repo_path / 'models/superglue.py'}'")
    print(f"       Check the file structure and class name in the repository.")
    print(f"       Error details: {e}")
    # Raise error or allow proceeding but model loading will fail
    raise e # Stop execution if import fails

# --- Configuration ---
# Setup device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# SIFT Configuration (OpenCV)
# Need opencv-contrib-python installed for SIFT
try:
    NUM_SIFT_FEATURES = 512 # Limit features (adjust as needed, e.g., 2048, 4096)
    sift = cv2.SIFT_create(nfeatures=NUM_SIFT_FEATURES)
    print(f"SIFT detector created successfully (max features = {NUM_SIFT_FEATURES}). OpenCV version: {cv2.__version__}")
    print(f"SIFT detector created successfully. OpenCV version: {cv2.__version__}")
    # Example: You can customize SIFT parameters here if needed
    # sift = cv2.SIFT_create(nfeatures=0, nOctaveLayers=3, contrastThreshold=0.04, edgeThreshold=10, sigma=1.6)
except AttributeError:
    print("ERROR: cv2.SIFT_create() not found.")
    print("       This usually means you don't have 'opencv-contrib-python' installed.")
    print("       Please install it: pip install opencv-contrib-python")
    sift = None # Set to None to handle gracefully later
    raise AttributeError("cv2.SIFT_create() not found. Install opencv-contrib-python.")

# SuperGlue Configuration (for this specific repo)
# Using the weights path you provided
superglue_config = {
    'descriptor_dim': 128, # SIFT descriptor dimension is 128
    # --- Using the weights path you specified ---
    'weights_path': str(superglue_sift_repo_path / 'models/weights/superglue_outdoor.pth'),
    # ------------------------------------------
    # --- These parameters below are examples - VERIFY or REMOVE if not needed by this repo's SuperGlue class ---
    'keypoint_encoder': [32, 64, 128], # Example network layer sizes
    'GNN_layers': ['self', 'cross'] * 9, # Example GNN structure
    'sinkhorn_iterations': 100, # Example Sinkhorn iterations
    'match_threshold': 0.2, # Example matching threshold (tune later)
    # --- End of Example Parameters ---
}
print(f"\nUsing SuperGlue config:")
# Print config nicely
for key, value in superglue_config.items():
    # Shorten long GNN layer list for printing
    if key == 'GNN_layers' and isinstance(value, list) and len(value) > 6:
         print(f"  '{key}': {value[:3]}... (x{len(value)})")
    else:
         print(f"  '{key}': {value}")

# Check if the specified weights file exists
weights_file = Path(superglue_config['weights_path'])
if not weights_file.is_file():
     print(f"\n!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
     print(f"WARNING: Specified SuperGlue weights file NOT FOUND at:")
     print(f"         '{weights_file.resolve()}'")
     print(f"         Please ensure the file exists at this location.")
     print(f"         Model loading in Cell 5 will likely FAIL.")
     print(f"!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!\n")
else:
    print(f"\nSuperGlue weights file found at: {weights_file.resolve()}")
    # Add note about potential mismatch between weights and features
    if "superglue_outdoor" in weights_file.name or "superglue_indoor" in weights_file.name:
         print("(Note: These weights ('superglue_outdoor.pth') were likely trained on SuperPoint features, not SIFT.")
         print("       Matching performance may differ from the original SuperPoint results.)")


# --- Constants ---
QUERY_DIR = Path('./extended_dataset/camera image')
REF_DIR = Path('./extended_dataset/reference image')
REFERENCE_IMAGE_SIZE = (1000, 1000) # Keep assumption or load an image to check
VIZ_COUNT = 5 # Default number of results to visualize later

# Check if data directories exist early
if not QUERY_DIR.is_dir():
     print(f"\nWARNING: Query directory NOT FOUND at '{QUERY_DIR.resolve()}'")
if not REF_DIR.is_dir():
     print(f"\nWARNING: Reference directory NOT FOUND at '{REF_DIR.resolve()}'")

print("\nCell 1 Setup Complete.")

Script started/restarted: 2025-04-18 23:54:18
Found SuperGlue-pytorch repository at: /home/jain/Desktop/IITK/Project_EE798T/SuperGlue-pytorch
Successfully imported SuperGlue class from repository.
Using device: cuda
SIFT detector created successfully (max features = 512). OpenCV version: 4.11.0
SIFT detector created successfully. OpenCV version: 4.11.0

Using SuperGlue config:
  'descriptor_dim': 128
  'weights_path': SuperGlue-pytorch/models/weights/superglue_outdoor.pth
  'keypoint_encoder': [32, 64, 128]
  'GNN_layers': ['self', 'cross', 'self']... (x18)
  'sinkhorn_iterations': 100
  'match_threshold': 0.2

SuperGlue weights file found at: /home/jain/Desktop/IITK/Project_EE798T/SuperGlue-pytorch/models/weights/superglue_outdoor.pth
(Note: These weights ('superglue_outdoor.pth') were likely trained on SuperPoint features, not SIFT.
       Matching performance may differ from the original SuperPoint results.)

Cell 1 Setup Complete.


In [2]:
# Cell 2: Function to Parse Filename (No Changes Needed)
# Same function as before - parses ground truth from filename

def parse_filename(filename):
    """
    Parses the filename to extract ground truth coordinates and angle.
    Filename format: (ignore)_(x,y)_angle_(ignore).png
    Coordinates: (x, y) relative to reference center. +x down, +y right.
    Angle: Anti-clockwise rotation degrees (0-360).
    """
    try:
        # Using Path object's stem attribute is slightly cleaner
        fname_stem = filename.stem
        parts = fname_stem.split('_')
        if len(parts) < 3:
            # Handle cases like '.DS_Store' or other unexpected filenames gracefully
            if not fname_stem.startswith('.'):
                 print(f"Warning: Skipping poorly formatted file (less than 3 parts): {filename.name}")
            return None

        gt_x, gt_y, gt_angle = None, None, None
        coord_part_index, angle_part_index = -1, -1

        # Find coordinate part (flexible parsing)
        for i in range(1, len(parts) -1):
            part = parts[i]
            # Try regex for (float,float)
            coord_match = re.search(r'\((-?\d+\.?\d*),(-?\d+\.?\d*)\)', part)
            if coord_match:
                gt_x = float(coord_match.group(1))
                gt_y = float(coord_match.group(2))
                coord_part_index = i
                break
            # Try splitting by comma if no parentheses
            coord_parts = part.split(',')
            if len(coord_parts) == 2:
                 try:
                     gt_x_try = float(coord_parts[0])
                     gt_y_try = float(coord_parts[1])
                     # Basic sanity check for reasonable coordinate values if needed
                     # if -1000 < gt_x_try < 1000 and -1000 < gt_y_try < 1000:
                     gt_x = gt_x_try
                     gt_y = gt_y_try
                     coord_part_index = i
                     break
                 except ValueError:
                     continue # Not a float pair

        if gt_x is None:
             print(f"Warning: Could not parse coordinate part in file: {filename.name}")
             return None

        # Find angle part (usually follows coordinate part)
        # Start searching from the part *after* the identified coordinate part
        search_start_index = coord_part_index + 1
        for i in range(search_start_index, len(parts) -1):
             try:
                  gt_angle = float(parts[i])
                  angle_part_index = i
                  break # Found the first float after coordinates
             except ValueError:
                  continue

        if gt_angle is None:
            # Fallback: Search all parts between first and last underscore if not found after coords
             for i in range(1, len(parts) - 1):
                  if i == coord_part_index: continue # Skip the coord part itself
                  try:
                      gt_angle = float(parts[i])
                      angle_part_index = i
                      break
                  except ValueError:
                      continue

        if gt_angle is None:
             print(f"Warning: Could not parse angle part in file: {filename.name}")
             return None

        # Validate angle range (0-360) - allow slight tolerance
        if not (-1e-9 <= gt_angle <= 360 + 1e-9):
             print(f"Warning: Angle {gt_angle:.4f} out of range [0, 360] in file: {filename.name}. Adjusting.")
             gt_angle = gt_angle % 360
             if gt_angle < 0: gt_angle += 360 # Ensure positive

        # Optional: Validate coordinate ranges based on reference size if needed
        max_coord = REFERENCE_IMAGE_SIZE[0] / 2
        if not (-max_coord <= gt_x <= max_coord and -max_coord <= gt_y <= max_coord):
             print(f"Warning: Coords ({gt_x}, {gt_y}) seem out of expected range [+/-{max_coord}] for {filename.name}")
             # Decide whether to return None or proceed

        return {'x': gt_x, 'y': gt_y, 'angle': gt_angle}

    except Exception as e:
        print(f"Error parsing filename {filename.name}: {e}")
        # import traceback
        # traceback.print_exc() # Uncomment for detailed traceback during debugging
        return None

In [3]:
# Cell 3: Function to Estimate Transformation (No Changes Needed)
# This function works on keypoint coordinates (Nx2 numpy arrays) and matches,
# regardless of whether they came from SIFT or SuperPoint.

def estimate_transform(mkpts0, mkpts1):
    """
    Estimates rotation and translation from matched keypoint coordinates using RANSAC.
    Assumes mkpts0 and mkpts1 are Nx2 numpy arrays of corresponding points.

    Returns:
        - M: 2x3 Affine transformation matrix (or None if failed)
        - angle_deg: Estimated rotation in degrees (anti-clockwise, 0-360)
        - trans_px: Estimated translation (tx, ty) in pixels (relative to top-left)
        - mask: RANSAC inlier mask
    """
    if mkpts0 is None or mkpts1 is None or len(mkpts0) == 0 or len(mkpts1) == 0:
        # print("Warning: estimate_transform received empty keypoints.")
        return None, None, None, None

    min_req_matches = 3 # Minimum points for estimateAffine2D RANSAC
    if mkpts0.shape[0] < min_req_matches:
        # print(f"Warning: Not enough matches ({mkpts0.shape[0]}) for RANSAC.")
        return None, None, None, None

    # Ensure input arrays are float32, as expected by estimateAffine2D
    mkpts0 = np.float32(mkpts0)
    mkpts1 = np.float32(mkpts1)

    # Use cv2.estimateAffine2D for rotation, translation, and scale robustness
    M, mask = cv2.estimateAffine2D(mkpts0, mkpts1, method=cv2.RANSAC,
                                     ransacReprojThreshold=5.0, # Default threshold, might need tuning
                                     maxIters=2000,             # Default iterations
                                     confidence=0.99)           # Default confidence

    if M is None or mask is None:
        # print("Warning: RANSAC failed to find a transformation.")
        return None, None, None, None

    # Check number of inliers
    num_inliers = np.sum(mask)
    if num_inliers < min_req_matches:
        # print(f"Warning: Not enough RANSAC inliers ({num_inliers} < {min_req_matches})")
        return None, None, None, None

    # Decompose the Affine matrix M = [[a, b, tx], [c, d, ty]]
    # Extract translation
    tx = M[0, 2]
    ty = M[1, 2]
    trans_px_tl = (tx, ty)

    # Extract rotation angle: angle_rad = atan2(sin(A), cos(A)) = A
    angle_rad = math.atan2(M[1, 0], M[0, 0])
    angle_deg_raw = np.degrees(angle_rad)

    # Convert to anti-clockwise image rotation (0-360)
    angle_deg = angle_deg_raw % 360
    # Ensure positive range [0, 360)
    # if angle_deg < 0: angle_deg += 360 # Modulo handles negativity correctly in Python? Let's be safe.
    # Python's % operator preserves the sign of the divisor. For -10 % 360 -> 350. So it should be okay.

    return M, angle_deg, trans_px_tl, mask

In [4]:
# Cell 4: Function for Coordinate Conversion (No Changes Needed)
# Works on translation, angle, and shapes, independent of feature type.

def convert_to_relative_center_coords(trans_px_tl, angle_deg, query_shape, ref_shape):
    """
    Converts the translation relative to top-left corners and rotation
    to the translation relative to image centers, using the user's convention.

    Args:
        trans_px_tl (tuple): (tx, ty) translation from query top-left to ref top-left (OpenCV standard).
        angle_deg (float): Anti-clockwise rotation angle in degrees.
        query_shape (tuple): (height, width) of the query image.
        ref_shape (tuple): (height, width) of the reference image.

    Returns:
        tuple: (rel_x, rel_y) relative center coordinates (+x down, +y right).
               Returns (None, None) if input is invalid.
    """
    if trans_px_tl is None or angle_deg is None:
        return None, None

    q_h, q_w = query_shape[:2]
    r_h, r_w = ref_shape[:2]

    # Reference image center (in OpenCV coords: origin top-left, +x right, +y down)
    ref_center_cv_x = r_w / 2.0
    ref_center_cv_y = r_h / 2.0

    # Query image center (in its own local coords: origin top-left)
    query_center_local_x = q_w / 2.0
    query_center_local_y = q_h / 2.0

    # Reconstruct the affine matrix using the estimated components (assuming scale=1 implicitly from estimateAffine2D)
    # M already estimated by estimate_transform handles scale/shear if present
    angle_rad = np.radians(angle_deg)
    cos_a = np.cos(angle_rad)
    sin_a = np.sin(angle_rad)
    tx, ty = trans_px_tl

    # We need the matrix M that maps query points to ref points.
    # Let's assume estimate_transform returned the correct M for the transformation.
    # If M wasn't returned, we can reconstruct an approximate one (less accurate if there was scale/shear)
    # For now, let's assume we have M or reconstruct it simply.
    # Reconstructing simply (may be slightly inaccurate if estimate_transform result had scale/shear):
    M_approx = np.array([
        [cos_a, -sin_a, tx],
        [sin_a,  cos_a, ty]
    ])

    # Transform the query center using the affine matrix
    query_center_in_ref_x = M_approx[0, 0] * query_center_local_x + M_approx[0, 1] * query_center_local_y + M_approx[0, 2]
    query_center_in_ref_y = M_approx[1, 0] * query_center_local_x + M_approx[1, 1] * query_center_local_y + M_approx[1, 2]

    # Find the vector from the reference center to the query center (in OpenCV coords)
    vec_cv_x = query_center_in_ref_x - ref_center_cv_x
    vec_cv_y = query_center_in_ref_y - ref_center_cv_y

    # Convert this vector to the user's coordinate system (+x down, +y right)
    rel_x = vec_cv_y # User's x is OpenCV's y
    rel_y = vec_cv_x # User's y is OpenCV's x

    return rel_x, rel_y

In [5]:
# Cell 5: Load SuperGlue Model (SIFT Version - MODIFIED FOR float32)

# Ensure Path is available if running standalone after restart
from pathlib import Path
import torch # Ensure torch is available

# Assume superglue_config, device, superglue_sift_repo_path are defined from Cell 1
# Assume SuperGlue class is imported from Cell 1

print("Loading SuperGlue model (SIFT version, float32)...")

# Instantiate the SuperGlue model from the cloned repository
# Assumes 'SuperGlue' class was successfully imported in Cell 1
try:
    # Check if config exists (defined in Cell 1)
    if 'superglue_config' not in locals():
        raise NameError("superglue_config dictionary not found. Run Cell 1.")
    if 'device' not in locals():
         raise NameError("device variable not found. Run Cell 1.")

    weights_path = superglue_config['weights_path']
    if not Path(weights_path).is_file():
         raise FileNotFoundError(f"SuperGlue weights not found at {weights_path}")

    # Initialize model with config
    # Assumes SuperGlue class was imported correctly in Cell 1
    superglue = SuperGlue(superglue_config).eval().to(device)

    # --- NO .double() CALL HERE ---
    # This assumes you modified models/superglue.py to remove internal .double() calls

    print(f"SuperGlue model loaded successfully (intended for float32) to device: {device}.")

except FileNotFoundError as e:
     print(f"ERROR: {e}")
     print("Please ensure the weights path in superglue_config (Cell 1) is correct and the file exists.")
     superglue = None # Set to None to prevent errors later
except NameError as e:
     print(f"ERROR: A required variable from Cell 1 is missing: {e}")
     superglue = None
except AttributeError as e:
     # This might happen if SuperGlue class wasn't imported correctly in Cell 1
     print(f"ERROR: Could not find SuperGlue class or method. Import failed or class name incorrect?")
     if 'superglue_sift_repo_path' in locals():
          print(f"       (Looking in: {superglue_sift_repo_path})")
     print(f"       Error details: {e}")
     superglue = None
except Exception as e:
     print(f"ERROR: Failed to load SuperGlue model. Details: {e}")
     # import traceback
     # traceback.print_exc() # Uncomment for detailed traceback
     superglue = None

# SIFT detector check (assumes 'sift' was defined in Cell 1)
if 'sift' not in locals() or sift is None:
     print("ERROR: SIFT detector not available (check Cell 1).")
else:
     print("SIFT detector ready.")

print("\n--- Cell 5 Finished ---")

Loading SuperGlue model (SIFT version, float32)...
SuperGlue model loaded successfully (intended for float32) to device: cuda.
SIFT detector ready.

--- Cell 5 Finished ---


In [ ]:
# Cell 6: Main Processing Loop (SIFT Version - PROCESS ALL IMAGES)

import traceback # Import for detailed error printing
import time # Ensure time is available if not imported globally
# Ensure Path is imported if running this cell standalone after restart
from pathlib import Path

# Initialize lists and counters
results = []
all_filenames = []
processing_errors = 0

# Check if model and detector loaded successfully before starting
if 'superglue' not in locals() or 'sift' not in locals() or superglue is None or sift is None:
    print("\nERROR: SuperGlue model or SIFT detector not available (check Cell 1 and Cell 5).")
    print("Cannot proceed with processing loop.")
else:
    print(f"\nStarting image processing loop for ALL files at {time.strftime('%Y-%m-%d %H:%M:%S')}...")
    # Ensure directories exist
    if not QUERY_DIR.is_dir() or not REF_DIR.is_dir():
        print(f"ERROR: Query directory '{QUERY_DIR.resolve()}' or Reference directory '{REF_DIR.resolve()}' not found.")
    else:
        # Get list of query images, ignoring hidden files
        query_files = sorted([f for f in QUERY_DIR.iterdir() if f.is_file() and not f.name.startswith('.') and f.suffix.lower() == '.png'])

        if not query_files:
            print(f"No PNG files found in {QUERY_DIR}")
        else:
            total_files = len(query_files)
            print(f"Found {total_files} query images. Processing...")

            for i, query_path in enumerate(query_files):
                filename = query_path.name
                ref_path = REF_DIR / filename

                # Print progress periodically
                if (i + 1) % 50 == 0 or i == 0:
                    print(f"--- Processing file {i+1}/{total_files}: {filename} ---")

                if not ref_path.exists():
                    # print(f"  Warning: Reference image not found for {filename}. Skipping.") # Optional: uncomment for verbose skipping
                    # Add failure record for missing reference
                    results.append({'filename': filename, 'query_path': str(query_path), 'ref_path': str(ref_path), 'error': 'Reference Missing'})
                    all_filenames.append(filename)
                    processing_errors += 1
                    continue # Skip to the next file

                # 1. Parse Ground Truth
                ground_truth = parse_filename(query_path) # Use query path for parsing convention
                if ground_truth is None:
                    # print(f"  Warning: Failed to parse ground truth for {filename}. Skipping processing.") # Optional: uncomment
                    results.append({'filename': filename, 'query_path': str(query_path), 'ref_path': str(ref_path), 'error': 'Filename Parsing Error'})
                    all_filenames.append(filename)
                    processing_errors += 1
                    continue # Skip file if parsing failed

                # --- Start Try Block for Core Processing ---
                try:
                    # 2. Load Images (as grayscale for SIFT)
                    img0_bgr = cv2.imread(str(query_path), cv2.IMREAD_COLOR)
                    img1_bgr = cv2.imread(str(ref_path), cv2.IMREAD_COLOR)
                    if img0_bgr is None or img1_bgr is None: raise ValueError("Image loading failed")
                    img0_gray = cv2.cvtColor(img0_bgr, cv2.COLOR_BGR2GRAY)
                    img1_gray = cv2.cvtColor(img1_bgr, cv2.COLOR_BGR2GRAY)
                    query_h, query_w = img0_gray.shape[:2]
                    ref_h, ref_w = img1_gray.shape[:2]

                    # 3. Detect SIFT features and compute descriptors
                    # Using sift object defined in Cell 1 (potentially with nfeatures limit)
                    kp0_sift, desc0_sift = sift.detectAndCompute(img0_gray, None)
                    kp1_sift, desc1_sift = sift.detectAndCompute(img1_gray, None)
                    num_kp0 = len(kp0_sift) if kp0_sift is not None else 0
                    num_kp1 = len(kp1_sift) if kp1_sift is not None else 0
                    if desc0_sift is None or desc1_sift is None or num_kp0 == 0 or num_kp1 == 0: raise ValueError(f"SIFT Failure (KP0={num_kp0}, KP1={num_kp1})")

                    # 4. Prepare data for SuperGlue (with workarounds)
                    kpts0 = np.array([kp.pt for kp in kp0_sift], dtype=np.float32)
                    kpts1 = np.array([kp.pt for kp in kp1_sift], dtype=np.float32)
                    scores0 = np.array([kp.response for kp in kp0_sift], dtype=np.float32)
                    scores1 = np.array([kp.response for kp in kp1_sift], dtype=np.float32)
                    desc0 = desc0_sift.T; desc1 = desc1_sift.T
                    kpts0_tensor = torch.from_numpy(kpts0).unsqueeze(0).to(device)
                    kpts1_tensor = torch.from_numpy(kpts1).unsqueeze(0).to(device)
                    desc0_tensor = torch.from_numpy(desc0).unsqueeze(0).to(device)
                    desc1_tensor = torch.from_numpy(desc1).unsqueeze(0).to(device)
                    scores0_tensor = torch.from_numpy(scores0).unsqueeze(1).to(device) # Shape: (N, 1) - Workaround!
                    scores1_tensor = torch.from_numpy(scores1).unsqueeze(1).to(device) # Shape: (N, 1) - Workaround!
                    img0_size = torch.tensor([[query_h, query_w]], dtype=torch.float32, device=device)
                    img1_size = torch.tensor([[ref_h, ref_w]], dtype=torch.float32, device=device)
                    image0_tensor = torch.from_numpy(img0_gray).float()[None, None].to(device) / 255.0
                    image1_tensor = torch.from_numpy(img1_gray).float()[None, None].to(device) / 255.0

                    pred_input = {
                        'keypoints0': kpts0_tensor, 'scores0': scores0_tensor, 'descriptors0': desc0_tensor, 'image_size0': img0_size, 'image0': image0_tensor,
                        'keypoints1': kpts1_tensor, 'scores1': scores1_tensor, 'descriptors1': desc1_tensor, 'image_size1': img1_size, 'image1': image1_tensor,
                        'file_name': filename,
                        'all_matches': torch.empty((1, 0, 2), dtype=torch.long).to(device), # Placeholder
                    }

                    # 5. Perform SuperGlue Matching
                    with torch.no_grad():
                        pred = superglue(pred_input) # Model call

                    # 6. Extract Matches and Confidences
                    matches0 = pred.get('matches0', [None])[0]; match_conf = pred.get('matching_scores0', [None])[0]
                    if matches0 is None or match_conf is None: raise ValueError("SuperGlue Output Format Error")
                    matches0 = matches0.cpu().numpy(); match_conf = match_conf.cpu().numpy()

                    # Filter matches
                    valid_match_mask = (matches0 > -1) & (match_conf > superglue_config['match_threshold'])
                    match_indices0 = np.where(valid_match_mask)[0]
                    num_good_matches = len(match_indices0)

                    # Get coordinates
                    mkpts0 = kpts0[match_indices0]
                    # Important fix: Ensure indices used for kpts1 are valid
                    valid_matches1 = matches0[valid_match_mask]
                    if np.any(valid_matches1 >= len(kpts1)): # Check if any index is out of bounds for kpts1
                        raise IndexError(f"Invalid match index found for kpts1 (max index: {len(kpts1)-1}, found: {valid_matches1.max()})")
                    mkpts1 = kpts1[valid_matches1]


                    # 7. Estimate Transformation
                    M, pred_angle, trans_px_tl, mask = estimate_transform(mkpts0, mkpts1) # Uses Cell 3 function

                    # 8. Convert Coordinates
                    if M is not None:
                        pred_x, pred_y = convert_to_relative_center_coords(trans_px_tl, pred_angle, (query_h, query_w), (ref_h, ref_w))
                    else:
                        pred_x, pred_y, pred_angle = None, None, None # Prediction failed

                    # 9. Store Results
                    results.append({
                        'filename': filename, 'query_path': str(query_path), 'ref_path': str(ref_path),
                        'gt_x': ground_truth.get('x'), 'gt_y': ground_truth.get('y'), 'gt_angle': ground_truth.get('angle'),
                        'pred_x': pred_x, 'pred_y': pred_y, 'pred_angle': pred_angle,
                        'query_shape': (query_h, query_w), 'ref_shape': (ref_h, ref_w),
                        'error': None # No error in this try block path
                    })
                    all_filenames.append(filename)

                # --- End Try Block ---
                except Exception as e:
                    processing_errors += 1
                    # Reduce print frequency for errors during full run unless debugging needed
                    if processing_errors < 10 or (i+1)%50==0: # Print first few errors and periodically
                         print(f"!! Error processing file {filename} (Error #{processing_errors}): {e}")
                         # print("\n--- TRACEBACK ---"); traceback.print_exc(); print("--- END TRACEBACK ---\n") # Keep commented unless debugging
                    # Store failure information
                    results.append({
                        'filename': filename, 'query_path': str(query_path), 'ref_path': str(ref_path),
                        'gt_x': ground_truth.get('x'), 'gt_y': ground_truth.get('y'), 'gt_angle': ground_truth.get('angle'),
                        'pred_x': None, 'pred_y': None, 'pred_angle': None,
                        'error': str(e)
                    })
                    all_filenames.append(filename)

            # --- End For Loop ---
            print(f"\nProcessing finished at {time.strftime('%Y-%m-%d %H:%M:%S')}.")
            successful_run_count = len(results) - processing_errors # Approximation, as errors might happen before appending
            print(f"Attempted {len(all_filenames)} files.") # Use all_filenames length for total attempts
            print(f"Entries added to results list: {len(results)}")
            print(f"Encountered {processing_errors} errors during processing loop.")

# --- End of Cell 6 ---
print("\n--- Cell 6 Finished (Full Dataset Mode) ---")


Starting image processing loop for ALL files at 2025-04-18 23:54:18...
Found 2040 query images. Processing...
--- Processing file 1/2040: (10002,10091)_(-404.91,33.19)_145.08_mumbai.png ---
!! Error processing file (10002,10091)_(-404.91,33.19)_145.08_mumbai.png (Error #1): CUDA out of memory. Tried to allocate 516.00 MiB. GPU 0 has a total capacity of 3.81 GiB of which 310.38 MiB is free. Process 15497 has 48.00 MiB memory in use. Including non-PyTorch memory, this process has 3.45 GiB memory in use. Of the allocated memory 3.04 GiB is allocated by PyTorch, and 347.19 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)
!! Error processing file (10004,13940)_(-316.73,-56.63)_267.36_mumbai.png (Error #2): CUDA out of memory. Tried to allocate 514.00 MiB. 

In [ ]:
# Cell 7: Calculate Accuracy Metrics (No Changes Needed)
# Same code as before, calculates metrics based on the 'results' list.

coord_errors = []
angle_errors = []
successful_predictions = 0

print("\nCalculating accuracy metrics...")
if not results:
     print("No results to analyze.")
else:
    for res in results:
        # Check if prediction fields exist and are not None
        if res.get('pred_x') is not None and res.get('pred_y') is not None and res.get('pred_angle') is not None:
            successful_predictions += 1

            # Coordinate Error (Euclidean Distance)
            dx = res['pred_x'] - res['gt_x']
            dy = res['pred_y'] - res['gt_y']
            coord_err = np.sqrt(dx**2 + dy**2)
            coord_errors.append(coord_err)

            # Angle Error (handle wrap-around)
            angle_diff = abs(res['pred_angle'] - res['gt_angle'])
            angle_err = min(angle_diff, 360.0 - angle_diff)
            angle_errors.append(angle_err)
        # else: prediction failed or errored, skip metrics for this entry

    print(f"\n--- Accuracy Metrics ({successful_predictions} successful predictions out of {len(results)} total entries) ---")
    if successful_predictions > 0:
        mean_coord_error = np.mean(coord_errors)
        median_coord_error = np.median(coord_errors)
        std_coord_error = np.std(coord_errors)

        mean_angle_error = np.mean(angle_errors)
        median_angle_error = np.median(angle_errors)
        std_angle_error = np.std(angle_errors)

        print(f"Coordinate Error (pixels):")
        print(f"  Mean:   {mean_coord_error:.3f}")
        print(f"  Median: {median_coord_error:.3f}")
        print(f"  StdDev: {std_coord_error:.3f}")

        print(f"\nAngle Error (degrees):")
        print(f"  Mean:   {mean_angle_error:.3f}")
        print(f"  Median: {median_angle_error:.3f}")
        print(f"  StdDev: {std_angle_error:.3f}")

        # Optional: Histograms
        # plt.figure(figsize=(12, 5))
        # plt.subplot(1, 2, 1); plt.hist(coord_errors, bins=50); plt.title('Coordinate Error Distribution'); plt.xlabel('Error (pixels)'); plt.ylabel('Frequency')
        # plt.subplot(1, 2, 2); plt.hist(angle_errors, bins=50); plt.title('Angle Error Distribution'); plt.xlabel('Error (degrees)'); plt.ylabel('Frequency')
        # plt.tight_layout(); plt.show()

    else:
        print("No successful predictions were made to calculate metrics.")

    total_processed = len(results) # Number of entries in results list
    failure_rate = ((total_processed - successful_predictions) / total_processed * 100) if total_processed > 0 else 0
    print(f"\nTotal entries processed: {total_processed}")
    print(f"Prediction success rate: {100 - failure_rate:.2f}%")
    print(f"Prediction failure/error rate: {failure_rate:.2f}%")

In [ ]:
# Cell 8: Visualization Function (No Changes Needed)
# Same function as before, uses image paths and predicted/GT coordinates.

def get_rotated_rect_corners(center_x, center_y, w, h, angle_deg, ref_shape):
    """
    Calculates the four corner points of a rectangle rotated around its center.
    Center coordinates are relative to the reference image center (+x down, +y right).
    Angle is anti-clockwise degrees.
    Returns corners in OpenCV format (integer tuples, origin top-left).
    """
    if None in [center_x, center_y, w, h, angle_deg, ref_shape] or None in ref_shape:
        print("Warning: Invalid input to get_rotated_rect_corners.")
        return None # Return None if any input is invalid

    r_h, r_w = ref_shape[:2]
    ref_center_cv_x = r_w / 2.0
    ref_center_cv_y = r_h / 2.0

    # Convert user center coords back to OpenCV top-left origin coords
    center_cv_x = center_y + ref_center_cv_x # User y is CV x
    center_cv_y = center_x + ref_center_cv_y # User x is CV y

    # Angle for rotation matrix (negative for rotating points)
    angle_rad = np.radians(-angle_deg)
    cos_a = np.cos(angle_rad)
    sin_a = np.sin(angle_rad)

    # Rectangle corners relative to its center (0,0)
    hw = w / 2.0
    hh = h / 2.0
    corners_local = np.array([
        [-hw, -hh], [ hw, -hh], [ hw,  hh], [-hw,  hh]
    ])

    # Rotate and translate corners
    corners_rotated = np.zeros_like(corners_local)
    for i, (x, y) in enumerate(corners_local):
        x_rot = x * cos_a - y * sin_a
        y_rot = x * sin_a + y * cos_a
        corners_rotated[i, 0] = x_rot + center_cv_x
        corners_rotated[i, 1] = y_rot + center_cv_y

    return corners_rotated.astype(int)


def visualize_result(result_data, border_thickness=3):
    """
    Visualizes the query, reference, and localization boxes (predicted vs ground truth).
    Handles cases where prediction failed (pred_x is None).
    """
    query_img_bgr = cv2.imread(result_data.get('query_path', ''))
    ref_img_bgr = cv2.imread(result_data.get('ref_path', ''))

    if query_img_bgr is None or ref_img_bgr is None:
        print(f"Error loading images for visualization: {result_data.get('filename', 'N/A')}")
        # Create placeholder images maybe? Or just don't plot.
        plt.figure(figsize=(15, 7))
        plt.text(0.5, 0.5, f"Error loading images for\n{result_data.get('filename', 'N/A')}",
                 horizontalalignment='center', verticalalignment='center')
        plt.title("Visualization Error")
        plt.show()
        return

    q_h, q_w = result_data.get('query_shape', (None, None))[:2]
    ref_shape = result_data.get('ref_shape', (None, None))

    if q_h is None or q_w is None or ref_shape[0] is None or ref_shape[1] is None:
        print(f"Error: Missing shape information for {result_data.get('filename', 'N/A')}")
        # Plot images without boxes?
        fig, axes = plt.subplots(1, 2, figsize=(15, 7))
        axes[0].imshow(cv2.cvtColor(query_img_bgr, cv2.COLOR_BGR2RGB)); axes[0].set_title("Query"); axes[0].axis('off')
        axes[1].imshow(cv2.cvtColor(ref_img_bgr, cv2.COLOR_BGR2RGB)); axes[1].set_title("Reference (Shape Error)"); axes[1].axis('off')
        plt.show()
        return


    # --- Draw Ground Truth Box (Green) ---
    # Ensure GT values are present before drawing
    gt_present = all(k in result_data and result_data[k] is not None for k in ['gt_x', 'gt_y', 'gt_angle'])
    if gt_present:
         gt_corners = get_rotated_rect_corners(
             result_data['gt_x'], result_data['gt_y'],
             q_w, q_h, result_data['gt_angle'], ref_shape
         )
         if gt_corners is not None:
              cv2.polylines(ref_img_bgr, [gt_corners], isClosed=True, color=(0, 255, 0), thickness=border_thickness) # Green
    else:
        print(f"Warning: Ground truth data missing for {result_data.get('filename', 'N/A')}. Skipping GT box.")


    # --- Draw Predicted Box (Red) ---
    pred_status = "Failed"
    error_text = result_data.get('error', "Prediction Failed") # Use stored error if available
    pred_present = all(k in result_data and result_data[k] is not None for k in ['pred_x', 'pred_y', 'pred_angle'])

    if pred_present:
        pred_corners = get_rotated_rect_corners(
            result_data['pred_x'], result_data['pred_y'],
            q_w, q_h, result_data['pred_angle'], ref_shape
        )
        if pred_corners is not None:
             cv2.polylines(ref_img_bgr, [pred_corners], isClosed=True, color=(0, 0, 255), thickness=border_thickness) # Red
             pred_status = "Successful"
             # Calculate errors on the fly if not stored, requires GT to be present
             if gt_present:
                  dx = result_data['pred_x'] - result_data['gt_x']
                  dy = result_data['pred_y'] - result_data['gt_y']
                  coord_err = np.sqrt(dx**2 + dy**2)
                  angle_diff = abs(result_data['pred_angle'] - result_data['gt_angle'])
                  angle_err = min(angle_diff, 360.0 - angle_diff)
                  error_text = f"Coord Err: {coord_err:.2f}px, Angle Err: {angle_err:.2f}deg"
             else:
                  error_text = "Pred OK, GT Missing"
    # else: pred_status remains "Failed", error_text uses default or stored error

    # --- Display using Matplotlib ---
    fig, axes = plt.subplots(1, 2, figsize=(15, 7))

    # Display Query Image
    axes[0].imshow(cv2.cvtColor(query_img_bgr, cv2.COLOR_BGR2RGB))
    axes[0].set_title(f"Query: {result_data.get('filename', 'N/A')}")
    axes[0].axis('off')

    # Display Reference Image with Boxes
    axes[1].imshow(cv2.cvtColor(ref_img_bgr, cv2.COLOR_BGR2RGB))
    axes[1].set_title(f"Reference (GT=Green, Pred=Red) - Status: {pred_status}")
    # Display error text calculated above or stored error message
    axes[1].text(5, 25, error_text, color='yellow', fontsize=9, bbox=dict(facecolor='black', alpha=0.6))
    axes[1].axis('off')

    # Format GT and Pred strings safely using .get with default values
    gt_x_str = f"{result_data.get('gt_x', 'N/A'):.2f}" if result_data.get('gt_x') is not None else 'N/A'
    gt_y_str = f"{result_data.get('gt_y', 'N/A'):.2f}" if result_data.get('gt_y') is not None else 'N/A'
    gt_a_str = f"{result_data.get('gt_angle', 'N/A'):.2f}" if result_data.get('gt_angle') is not None else 'N/A'
    gt_info = f"GT: ({gt_x_str}, {gt_y_str}), {gt_a_str}°"

    pred_x_str = f"{result_data.get('pred_x', 'N/A'):.2f}" if result_data.get('pred_x') is not None else 'N/A'
    pred_y_str = f"{result_data.get('pred_y', 'N/A'):.2f}" if result_data.get('pred_y') is not None else 'N/A'
    pred_a_str = f"{result_data.get('pred_angle', 'N/A'):.2f}" if result_data.get('pred_angle') is not None else 'N/A'
    pred_info = f"Pred: ({pred_x_str}, {pred_y_str}), {pred_a_str}°" if pred_status == "Successful" else f"Pred: {pred_status}"

    plt.suptitle(f"{gt_info}\n{pred_info}", y=0.95)

    plt.tight_layout(rect=[0, 0.03, 1, 0.9])
    plt.show()

In [ ]:
# Cell 8.5: Separate Results and Calculate Errors (SIFT Version)
# No changes needed from the version used previously, as it just processes the 'results' list.
# Ensure error calculation handles potential None values robustly.

successful_results = []
failed_results = []
high_error_results = [] # New list

# Define error thresholds (tune if needed)
COORD_ERROR_THRESHOLD = 50.0 # pixels
ANGLE_ERROR_THRESHOLD = 20.0 # degrees

print("\nSeparating results and calculating errors...")
# Check if 'results' variable exists and is a list
if 'results' not in locals() or not isinstance(results, list):
    print("="*60)
    print("ERROR: 'results' variable not found or is not a list.")
    print("Please ensure Cell 6 (Main Processing Loop) has been run.")
    print("="*60)
    # Define empty lists to prevent errors later
    successful_results = []
    failed_results = []
    high_error_results = []
else:
    # Iterate through the existing results list
    for res in results:
        # Check if prediction was successful (all prediction fields present and not None)
        is_successful = (res.get('pred_x') is not None and
                         res.get('pred_y') is not None and
                         res.get('pred_angle') is not None)

        if is_successful:
            # Ensure ground truth is also present for error calculation
            gt_present = (res.get('gt_x') is not None and
                          res.get('gt_y') is not None and
                          res.get('gt_angle') is not None)

            if gt_present:
                # Calculate errors
                dx = res['pred_x'] - res['gt_x']
                dy = res['pred_y'] - res['gt_y']
                coord_err = np.sqrt(dx**2 + dy**2)

                angle_diff = abs(res['pred_angle'] - res['gt_angle'])
                angle_err = min(angle_diff, 360.0 - angle_diff)

                # Store errors back into the dictionary
                res['coord_error'] = coord_err
                res['angle_error'] = angle_err

                # Check if it qualifies for high error list
                if coord_err > COORD_ERROR_THRESHOLD or angle_err > ANGLE_ERROR_THRESHOLD:
                    high_error_results.append(res)
            else:
                 # GT missing, cannot calculate error, assign None
                 res['coord_error'] = None
                 res['angle_error'] = None

            successful_results.append(res) # Add to successful list regardless of error calc

        else:
            # Failed prediction or processing error
            res['coord_error'] = None # No error to calculate
            res['angle_error'] = None
            failed_results.append(res)

    print(f"Separation and Error Calculation complete:")
    print(f"  Found {len(successful_results)} successful predictions.")
    print(f"  Found {len(failed_results)} failed predictions/errors.")
    print(f"  Identified {len(high_error_results)} successful predictions with high error (where calculable).")

    # Optional: Sort high_error_results if desired (e.g., by coordinate error descending)
    # Need to handle None values if sorting
    # high_error_results.sort(key=lambda x: x.get('coord_error', float('inf')), reverse=True)

In [ ]:
# Cell 9: Visualization Loops (SIFT Version)
# No changes needed from the version used previously, displays Failed, High-Error, and Low/Medium-Error.

MAX_VIZ_EACH = 5 # Number of examples to show for each category

# Check if prerequisite lists exist
if 'failed_results' not in locals() or 'high_error_results' not in locals() or 'successful_results' not in locals():
     print("ERROR: Results lists (failed, high_error, successful) not found. Please run Cell 8.5 first.")
else:
    # --- Visualize FAILED Predictions ---
    print(f"\n{'='*20} Visualizing up to {MAX_VIZ_EACH} FAILED Prediction Examples {'='*20}")
    if not failed_results:
        print("\nNo failed prediction results found to visualize.")
    else:
        print(f"\nDisplaying the first {min(len(failed_results), MAX_VIZ_EACH)} failed examples...")
        count_failed = 0
        for res in failed_results:
            if count_failed < MAX_VIZ_EACH:
                print(f"\n[{count_failed+1}/{min(len(failed_results), MAX_VIZ_EACH)}] Visualizing FAILED result for: {res.get('filename', 'N/A')}")
                try:
                    visualize_result(res)
                    count_failed += 1
                except Exception as e:
                     print(f"  ERROR during visualization for {res.get('filename', 'N/A')}: {e}")
            else:
                break # Stop after showing MAX_VIZ_EACH
        if count_failed == 0 and failed_results:
             print("\nCould not visualize any of the available failed results due to errors during visualization.")
        print(f"\nFinished displaying {count_failed} failed examples.")


    # --- Visualize HIGH ERROR Successful Predictions ---
    print(f"\n{'='*20} Visualizing up to {MAX_VIZ_EACH} HIGH ERROR Successful Prediction Examples {'='*20}")
    # Retrieve thresholds used in Cell 8.5 if they were defined there, otherwise redefine or display fixed values
    coord_thresh_disp = locals().get('COORD_ERROR_THRESHOLD', 50.0) # Get from locals or use default
    angle_thresh_disp = locals().get('ANGLE_ERROR_THRESHOLD', 20.0)
    print(f"(Thresholds used: Coord > {coord_thresh_disp}px OR Angle > {angle_thresh_disp}deg)")

    if not high_error_results:
        print("\nNo successful predictions with high error found to visualize.")
    else:
        print(f"\nDisplaying the first {min(len(high_error_results), MAX_VIZ_EACH)} high-error successful examples...")
        count_hierr = 0
        for res in high_error_results:
            if count_hierr < MAX_VIZ_EACH:
                coord_err_str = f"{res.get('coord_error', 'N/A'):.2f}" if res.get('coord_error') is not None else "N/A"
                angle_err_str = f"{res.get('angle_error', 'N/A'):.2f}" if res.get('angle_error') is not None else "N/A"
                print(f"\n[{count_hierr+1}/{min(len(high_error_results), MAX_VIZ_EACH)}] Visualizing HIGH ERROR result for: {res.get('filename', 'N/A')}")
                print(f"  Errors - Coord: {coord_err_str}px, Angle: {angle_err_str}deg")
                try:
                    visualize_result(res)
                    count_hierr += 1
                except Exception as e:
                     print(f"  ERROR during visualization for {res.get('filename', 'N/A')}: {e}")
            else:
                break
        if count_hierr == 0 and high_error_results:
             print("\nCould not visualize any of the available high-error results due to errors during visualization.")
        print(f"\nFinished displaying {count_hierr} high-error successful examples.")

    # --- Visualize GENERALLY Successful Predictions (filter out high errors) ---
    print(f"\n{'='*20} Visualizing up to {MAX_VIZ_EACH} LOW/MEDIUM Error Successful Prediction Examples {'='*20}")
    if not successful_results:
         print("\nNo successful predictions found at all.")
    else:
        # Filter out high-error examples already shown - use filename for robust check
        high_error_filenames = {res.get('filename') for res in high_error_results if res.get('filename')}
        low_med_error_results = [res for res in successful_results if res.get('filename') not in high_error_filenames]

        if not low_med_error_results:
             print("\nNo successful predictions with low/medium error found (all successful had high error or failed visualization).")
        else:
            print(f"\nDisplaying the first {min(len(low_med_error_results), MAX_VIZ_EACH)} low/medium-error successful examples...")
            count_success = 0
            for res in low_med_error_results:
                if count_success < MAX_VIZ_EACH:
                    coord_err_str = f"{res.get('coord_error', 'N/A'):.2f}" if res.get('coord_error') is not None else "N/A"
                    angle_err_str = f"{res.get('angle_error', 'N/A'):.2f}" if res.get('angle_error') is not None else "N/A"
                    print(f"\n[{count_success+1}/{min(len(low_med_error_results), MAX_VIZ_EACH)}] Visualizing LOW/MED ERROR result for: {res.get('filename', 'N/A')}")
                    print(f"  Errors - Coord: {coord_err_str}px, Angle: {angle_err_str}deg")
                    try:
                        visualize_result(res)
                        count_success += 1
                    except Exception as e:
                         print(f"  ERROR during visualization for {res.get('filename', 'N/A')}: {e}")
                else:
                    break
            if count_success == 0 and low_med_error_results:
                 print("\nCould not visualize any of the available low/medium-error results due to errors during visualization.")
            print(f"\nFinished displaying {count_success} low/medium-error successful examples.")

    print(f"\n{'='*60}")
    print("Visualization complete.")

In [ ]:
# Cell 10: Timing Prediction for Random Sample (SIFT Version)
# Uses SIFT feature extraction and the new SuperGlue model.

# Imports might be needed again if kernel restarted, but should be loaded
import time
import random
import numpy as np
import torch
import cv2
import math
from pathlib import Path
import traceback

# --- Configuration ---
NUM_SAMPLES_FOR_TIMING = 5
# Ensure these match paths used in Cell 1 and Cell 6
QUERY_DIR = Path('./extended_dataset/camera image')
REF_DIR = Path('./extended_dataset/reference image')

print(f"\n--- Timing Prediction (SIFT Version) for {NUM_SAMPLES_FOR_TIMING} Random Image Pairs ---")

# --- Pre-requisite Checks ---
# Check essential variables/functions defined in previous cells are available
required_vars_timing = ['sift', 'superglue', 'superglue_config', 'device', 'QUERY_DIR', 'REF_DIR']
required_funcs_timing = ['convert_to_relative_center_coords'] # Only this one is needed besides cv2/torch/np
missing_items_timing = []
for item_name in required_vars_timing + required_funcs_timing:
    if item_name not in locals():
         missing_items_timing.append(item_name)

if missing_items_timing:
    print("\nERROR: Missing required variables or functions for timing.")
    print(f"  Missing: {', '.join(missing_items_timing)}")
    print("  Please ensure Cells 1 and 5 (SIFT version) were run successfully IN THIS KERNEL SESSION.")
    print("\nCannot proceed with timing.")
else:
    # --- All checks passed, proceed with timing logic ---
    print("\n--- DEBUG: Pre-requisite checks passed. Proceeding with timing logic ---")
    # Use the variables confirmed to be in the local scope
    sift = locals()['sift']
    superglue = locals()['superglue']
    superglue_config = locals()['superglue_config']
    device = locals()['device']
    convert_to_relative_center_coords = locals()['convert_to_relative_center_coords']

    try:
        # 1. Get list of valid image pairs
        if not QUERY_DIR.is_dir() or not REF_DIR.is_dir():
             raise FileNotFoundError(f"Query ({QUERY_DIR}) or Reference ({REF_DIR}) image directory not found.")

        all_query_files = sorted([
            f for f in QUERY_DIR.iterdir()
            if f.is_file() and not f.name.startswith('.') and f.suffix.lower() == '.png'
        ])
        valid_pairs = [q for q in all_query_files if (REF_DIR / q.name).exists()]

        if not valid_pairs:
             print("No valid image pairs found in the dataset to perform timing.")
             num_to_process = 0
        else:
             num_to_process = min(len(valid_pairs), NUM_SAMPLES_FOR_TIMING)

        if num_to_process > 0:
            # 2. Select random sample
            selected_files = random.sample(valid_pairs, num_to_process)
            print(f"Selected {num_to_process} files for timing:")
            for f in selected_files: print(f"  - {f.name}")

            # 3. Run predictions and time
            start_time = time.time()
            processed_count = 0
            prediction_times = []

            for query_path in selected_files:
                filename = query_path.name
                ref_path = REF_DIR / filename
                pair_start_time = time.time()

                try:
                    # --- Core Prediction Logic (SIFT Version) ---
                    # a) Load Images (Grayscale)
                    img0_gray = cv2.imread(str(query_path), cv2.IMREAD_GRAYSCALE)
                    img1_gray = cv2.imread(str(ref_path), cv2.IMREAD_GRAYSCALE)
                    if img0_gray is None or img1_gray is None:
                        print(f"  Skipping {filename}: Grayscale image loading error.")
                        continue
                    query_h, query_w = img0_gray.shape[:2]
                    ref_h, ref_w = img1_gray.shape[:2]

                    # b) SIFT Feature Extraction
                    kp0_sift, desc0_sift = sift.detectAndCompute(img0_gray, None)
                    kp1_sift, desc1_sift = sift.detectAndCompute(img1_gray, None)
                    if desc0_sift is None or desc1_sift is None or len(kp0_sift) == 0 or len(kp1_sift) == 0:
                         print(f"  Skipping {filename}: SIFT found no keypoints/descriptors.")
                         continue

                    # c) Prepare data for SuperGlue (Match Cell 6 Logic)
                    kpts0 = np.array([kp.pt for kp in kp0_sift], dtype=np.float32)
                    kpts1 = np.array([kp.pt for kp in kp1_sift], dtype=np.float32)
                    scores0 = np.array([kp.response for kp in kp0_sift], dtype=np.float32)
                    scores1 = np.array([kp.response for kp in kp1_sift], dtype=np.float32)
                    # Use 1.0 if response is 0? Or normalize? Keep response for now.
                    # scores0 = np.maximum(scores0, 1e-6) # Avoid zero scores if they cause issues
                    # scores1 = np.maximum(scores1, 1e-6)

                    pred_input = {
                        'keypoints0': torch.from_numpy(kpts0).unsqueeze(0).to(device),
                        'keypoints1': torch.from_numpy(kpts1).unsqueeze(0).to(device),
                        'descriptors0': torch.from_numpy(desc0_sift.T).unsqueeze(0).to(device),
                        'descriptors1': torch.from_numpy(desc1_sift.T).unsqueeze(0).to(device),
                        'scores0': torch.from_numpy(scores0).unsqueeze(0).to(device),
                        'scores1': torch.from_numpy(scores1).unsqueeze(0).to(device),
                        'image_size0': torch.tensor([query_h, query_w], dtype=torch.float32, device=device).unsqueeze(0),
                        'image_size1': torch.tensor([ref_h, ref_w], dtype=torch.float32, device=device).unsqueeze(0)
                        # Add 'image0'/'image1' tensors if the model requires them
                    }

                    # d) SuperGlue Matching
                    with torch.no_grad():
                        pred = superglue(pred_input)

                    # e) Extract Matches & Confidence
                    matches0 = pred.get('matches0', [None])[0] # Handle missing key + batch dim
                    match_conf = pred.get('matching_scores0', [None])[0]
                    if matches0 is None or match_conf is None:
                         print(f"  Skipping {filename}: SuperGlue output format unexpected.")
                         continue
                    matches0 = matches0.cpu().numpy()
                    match_conf = match_conf.cpu().numpy()

                    # f) Filter Matches
                    valid_match_mask = (matches0 > -1) & (match_conf > superglue_config['match_threshold'])
                    match_indices0 = np.where(valid_match_mask)[0]
                    match_indices1 = matches0[valid_match_mask]
                    mkpts0 = kpts0[match_indices0]
                    mkpts1 = kpts1[match_indices1]

                    # g) Estimate Transformation
                    min_req_points = 3
                    if len(mkpts0) < min_req_points:
                         print(f"  Skipping {filename}: Not enough confident matches ({len(mkpts0)} < {min_req_points}).")
                         continue
                    M, mask = cv2.estimateAffine2D(np.float32(mkpts0), np.float32(mkpts1), method=cv2.RANSAC, ransacReprojThreshold=5.0)

                    # h) Check RANSAC & Decompose
                    estimation_succeeded = False
                    pred_angle = None
                    trans_px_tl = None
                    if M is not None:
                        num_inliers = np.sum(mask) if mask is not None else 0
                        if num_inliers >= min_req_points:
                            estimation_succeeded = True
                            tx, ty = M[0, 2], M[1, 2]
                            trans_px_tl = (tx, ty)
                            angle_rad = math.atan2(M[1, 0], M[0, 0])
                            pred_angle = np.degrees(angle_rad) % 360
                        else:
                             print(f"  Skipping {filename}: RANSAC found only {num_inliers} inliers.")
                    else:
                         print(f"  Skipping {filename}: estimateAffine2D failed.")

                    if not estimation_succeeded:
                         continue

                    # i) Convert Coordinates
                    pred_x, pred_y = convert_to_relative_center_coords(
                        trans_px_tl, pred_angle, (query_h, query_w), (ref_h, ref_w)
                    )
                    # --- End Core Prediction Logic ---

                    # Success for this pair
                    pair_end_time = time.time()
                    prediction_times.append(pair_end_time - pair_start_time)
                    processed_count += 1

                except Exception as e:
                    print(f"  ERROR processing {filename} during timing loop:")
                    print(traceback.format_exc())

            end_time = time.time()

            # 4. Calculate and report times
            total_wall_time = end_time - start_time
            if processed_count > 0:
                 total_prediction_time = sum(prediction_times)
                 average_time = total_prediction_time / processed_count
                 print(f"\n--- Timing Results (SIFT Version) ---")
                 print(f"Successfully processed {processed_count} out of {num_to_process} selected pairs.")
                 print(f"Total prediction time (sum of successful pairs): {total_prediction_time:.3f} seconds")
                 print(f"Average prediction time per successful pair:   {average_time:.3f} seconds")
                 print(f"Total wall time for {num_to_process} attempts: {total_wall_time:.3f} seconds")
            else:
                 print("\nNo pairs were successfully processed during timing.")
                 print(f"Total wall time for {num_to_process} attempts: {total_wall_time:.3f} seconds")

    except FileNotFoundError as fnf:
        print(f"ERROR during file operations: {fnf}")
    except NameError as ne:
         print(f"UNEXPECTED ERROR: A required variable is not defined ({ne}).")
    except Exception as e:
        print(f"An unexpected error occurred OUTSIDE the timing loop:")
        print(traceback.format_exc())

print("\n--- End of Cell 10 execution ---")